In [15]:
import requests
import os



# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = os.getenv("API_KEY")

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'uk' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h,spreads' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unixzz


In [16]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports/odds', 
    params={
        'api_key': API_KEY
    }
)


if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())



Failed to get sports: status_code 404, response body <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



In [17]:

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Number of events: 51
[{'id': 'f162eb479dcc497d078326a73771826a', 'sport_key': 'tennis_atp_paris_masters', 'sport_title': 'ATP Paris Masters', 'commence_time': '2025-10-25T09:00:00Z', 'home_team': 'Tomas Martin Etcheverry', 'away_team': 'Pedro Martinez', 'bookmakers': [{'key': 'coral', 'title': 'Coral', 'last_update': '2025-10-25T16:30:54Z', 'markets': [{'key': 'h2h', 'last_update': '2025-10-25T16:30:54Z', 'outcomes': [{'name': 'Pedro Martinez', 'price': 23.0}, {'name': 'Tomas Martin Etcheverry', 'price': 1.01}]}, {'key': 'spreads', 'last_update': '2025-10-25T16:30:54Z', 'outcomes': [{'name': 'Pedro Martinez', 'price': 1.55, 'point': 7.5}, {'name': 'Tomas Martin Etcheverry', 'price': 2.3, 'point': -7.5}]}]}, {'key': 'ladbrokes_uk', 'title': 'Ladbrokes', 'last_update': '2025-10-25T16:31:07Z', 'markets': [{'key': 'h2h', 'last_update': '2025-10-25T16:31:07Z', 'outcomes': [{'name': 'Pedro Martinez', 'price': 23.0}, {'name': 'Tomas Martin Etcheverry', 'price': 1.01}]}, {'key': 'spreads', 'la

In [18]:
import pandas as pd

# Convert the odds data to a CSV format
# We'll flatten the nested structure to create a comprehensive CSV

rows = []

for event in odds_json:
    event_id = event['id']
    sport_key = event['sport_key']
    sport_title = event['sport_title']
    commence_time = event['commence_time']
    home_team = event['home_team']
    away_team = event['away_team']
    
    # Loop through each bookmaker
    for bookmaker in event['bookmakers']:
        bookmaker_key = bookmaker['key']
        bookmaker_title = bookmaker['title']
        bookmaker_last_update = bookmaker['last_update']
        
        # Loop through each market (h2h, spreads, etc.)
        for market in bookmaker['markets']:
            market_key = market['key']
            market_last_update = market['last_update']
            

            for outcome in market['outcomes']:
                row = {
                    'event_id': event_id,
                    'sport_key': sport_key,
                    'sport_title': sport_title,
                    'commence_time': commence_time,
                    'home_team': home_team,
                    'away_team': away_team,
                    'bookmaker_key': bookmaker_key,
                    'bookmaker_title': bookmaker_title,
                    'bookmaker_last_update': bookmaker_last_update,
                    'market_key': market_key,
                    'market_last_update': market_last_update,
                    'outcome_name': outcome['name'],
                    'outcome_price': outcome['price'],
                    'outcome_point': outcome.get('point', None)  # spreads have points, h2h doesn't
                }
                rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Save to CSV
csv_filename = 'odds_data.csv'
df.to_csv(csv_filename, index=False)

print(f"Data saved to {csv_filename}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head(10))

Data saved to odds_data.csv

DataFrame shape: (1682, 14)

First few rows:
                           event_id                 sport_key  \
0  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
1  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
2  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
3  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
4  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
5  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
6  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
7  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
8  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   
9  f162eb479dcc497d078326a73771826a  tennis_atp_paris_masters   

         sport_title         commence_time                home_team  \
0  ATP Paris Masters  2025-10-25T09:00:00Z  Tomas Martin Etcheverry   
1  ATP Paris Masters  2025-10-25T09:00:00Z  Tomas Martin Etcheverry 